# Text to Cypher

## Installation

This notebook requires the following dependencies:

In [ ]:
%pip install neo4j-graphrag python-dotenv

## Connecting to Neo4j

The following cell creates an instance of the Neo4j Python Driver that the retrievers require to connect to the database.  The driver is created with environment variables set in your `.env` file.


In [41]:
%load_ext dotenv
%dotenv

from os import getenv

NEO4J_URL = getenv("NEO4J_URI") or "neo4j://localhost:7687"
NEO4J_USERNAME = getenv("NEO4J_USERNAME") or "neo4j"
NEO4J_PASSWORD = getenv("NEO4J_PASSWORD") or "neoneoneo"
NEO4J_DATABASE = getenv("NEO4J_DATABASE") or "neo4j"

from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    NEO4J_URL,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

driver.verify_connectivity() # Throws an error if the connection is not successful


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Text to Cypher

Convert natural language queries into Cypher statements and execute them against the database to get the answer.

### Obtaining the database schema

The Neo4j GraphRAG library provides two functions for retrieving the schema of an existing graph database.

In [ ]:
from neo4j_graphrag.schema import get_schema

schema = get_schema(driver, database=NEO4J_DATABASE)

print(schema)

### Creating a retriever

The `Text2CypherRetriever` retriever will use this schema to generate a Cypher statement

In [43]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import Text2CypherRetriever

llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

text2cypher_retriever = Text2CypherRetriever(
    llm=llm,
    driver=driver,
    neo4j_database=NEO4J_DATABASE,
    neo4j_schema=schema
)

result = text2cypher_retriever.search("What are the top risk factors that APPLE INC faces?")

In [ ]:
for item in result.items:
    print(item.content)


The `GraphRAG` class creates a retrieval pipeline that accepts a user input, uses a retriever to fetch the context, and uses an LLM to generate an answer.

In [ ]:
from neo4j_graphrag.generation import GraphRAG

# Instantiate the RAG pipeline
rag = GraphRAG(
    retriever=text2cypher_retriever,
    llm=llm
)

# Query the graph
query = "What are the top risk factors that APPLE INC faces?"

text2cypher_response = rag.search(query_text=query, return_context=True)

print(text2cypher_response.answer)


In [ ]:
print(text2cypher_response.retriever_result.metadata['cypher'])

for item in text2cypher_response.retriever_result.items:
    print(item.content)


## Demo datasets

Try running text-to-Cypher on a [demo dataset hosted on demo.neo4jlabs.com](https://demo.neo4jlabs.com/).  Use `neo4j+s://demo.neo4jlabs.com` as the connection URL and the dataset name as the username, password and database:

* `recommendations`
* `movies`
* `northwind`
* `fincen`
* `twitter`
* `stackoverflow`
* `gameofthrones`
* `gameofthrones`
* `neoflix`
* `wordnet`
* `slack`



In [47]:
DEMO = 'northwind'

got_driver = GraphDatabase.driver(
    "neo4j+s://demo.neo4jlabs.com",
    auth=(DEMO, DEMO)
)

got_driver.verify_connectivity()


In [48]:
got_text2cypher_retriever = Text2CypherRetriever(
    llm=llm,
    driver=got_driver,
    neo4j_database=DEMO,
    neo4j_schema=get_schema(got_driver, database=DEMO)
)

got_result = got_text2cypher_retriever.search("What are the top grossing products?")

In [ ]:
print(got_result.metadata['cypher'])

In [ ]:
for item in got_result.items:
    print(item.content)